<a href="https://colab.research.google.com/github/frankwillard/NBA-Hall-Of-Fame-Model/blob/main/Player_Web_Scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import string
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np

In [5]:
#Must decide variables

#Accolades:
#All_Star, MVP, All_NBA_1, All_NBA_2, All_NBA_3, DPOY, Champion
#Finals_MVP, All_D_1, All_D_2
#PTS_Champ, STL_Champ, REB_Champ, AST_Champ, BLK_Champ
#ROTY

#Average and Career Stats
#Pts, Reb, Assists, Steals, Blocks, FG%, 3P%, 3PM

#PER, VORP, BPM, WS, WS/48

#VORP_1?, WS_1?, PER_1?

In [ ]:
#Could always add MVP vote shares, rankings in stats within a season, playoff performance

In [ ]:
def scrape_player_data():

  letters = list(string.ascii_lowercase)

  for letter in letters:

    letter_url = f"https://www.basketball-reference.com/players/{letter}/"

    letter_html = urlopen(letter_url)

    letter_soup = BeautifulSoup(letter_html, features="lxml")

    #Find all players at url

    ths = letter_soup.findAll('tbody')[0].findAll('th')
    player_dict = {th.getText():th.find('a')['href'] for th in ths}
    #dict((th.getText(),th.find('a')['href'] ) for th in ths)
    #href_list = [th.find('a')['href'] for th in ths]

    for player in player_dict:

      href = player_dict[player]

      player_url = f"https://www.basketball-reference.com/{href}"

      player_html = urlopen(player_url)

      player_soup = BeautifulSoup(player_html, features="lxml")

      #Find all players at url


      #Accolades
      #May need something for if this is non-existent
      #Will also have to base it off of what is there- things there will vary
      #bling = soup.find(id='bling')


      #Per Game
      #per_game = soup.find(id='per_game')
      #per_game_row = per_game.findAll('tfoot')[0].findAll('tr')[0]

      #Totals
      #totals = soup.find(id='totals')
      #career_totals_row = totals.findAll('tfoot')[0].findAll('tr')[0]

      #Advanced
      #advanced = soup.find(id='advanced')
      #advanced_row = advanced.findAll('tfoot')[0].findAll('tr')[0]

      #Eligibility- If ended after 2018 then can join 2022
      #Use last year to determine if "active" (ineligible recently retired will be active)
      #Active will be train/val and test split



In [44]:
"""
If something wrong with tfoot- manually finds row from bottom

career_index = -1
trs_reversed = per_game.findAll('tr')
trs_reversed.reverse()

for i, tr in enumerate(trs_reversed):
  try:
    th = tr.findAll('th')[0].getText()
    if th == 'Career':
        career_index = i
        break
  except:
    print("No th found for column")

print(career_index)
#.findAll('th')[0].getText()

#Get Career
"""

3
